## Machine Learning 프로젝트 수행을 위한 코드 구조화

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [25]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [26]:
# 데이터를 불러옵니다.
base_path = '../../data/'
train = pd.read_csv(base_path + 'train.csv')
test = pd.read_csv(base_path + 'test.csv')
submission = pd.read_csv(base_path + 'sample_submission.csv')
print(train.shape, test.shape, submission.shape)

(101763, 23) (67842, 22) (67842, 2)


## 2. EDA

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

In [27]:
train.columns

Index(['id', 'loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e',
       'b', 't', 'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment',
       'uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount',
       'defects'],
      dtype='object')

### 3. 전처리

#### 결측치 처리

### 4. 학습 데이터 분할

In [28]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
from sklearn.model_selection import train_test_split

X = train.drop(columns=['defects'])
y = train.defects

# for OOF-prediction split 5% of data as validation dataset.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=61, stratify=y)

In [29]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)
print(y_train.mean(), y_val.mean())

(81410, 22) (81410,) (20353, 22) (20353,)
0.2266429185603734 0.22664963396059548


### 5. 학습 및 평가

In [30]:
# 간단하게 LightGBM 테스트
# 적당한 hyper-parameter 조합을 두었습니다. (항상 best는 아닙니다. 예시입니다.)
model = LGBMClassifier(
    n_jobs=-1,
    random_state=61
)

In [31]:
print("\nFitting LightGBM...")
model.fit(X_train, y_train)


Fitting LightGBM...


LGBMClassifier(random_state=61)

In [32]:
# metric은 그때마다 맞게 바꿔줘야 합니다.
evaluation_metric = roc_auc_score

In [33]:
print("Prediction")
pred_train = model.predict(X_train)
pred_val = model.predict(X_val)


train_score = evaluation_metric(y_train, pred_train)
val_score = evaluation_metric(y_val, pred_val)

print("Train Score : %.4f" % train_score)
print("Validation Score : %.4f" % val_score)

Prediction
Train Score : 0.6770
Validation Score : 0.6676


### 6. Hyper-parameter Tuning

In [34]:
def optimizer(trial, X, y, K):
    # 조절할 hyper-parameter 조합을 적어줍니다.
    max_depth = trial.suggest_int('max_depth', 15, 25)
    num_leaves = trial.suggest_categorical('num_leaves', [32, 64,128,256,512,1024,2048])
    min_child_samples = trial.suggest_int('min_child_samples', 60, 80)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 0.8)
    n_estimators = trial.suggest_int('n_estimators', 50, 2047)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.1)

    # 원하는 모델을 지정합니다, optuna는 시간이 오래걸리기 때문에 저는 보통 RF로 일단 테스트를 해본 뒤에 LGBM을 사용합니다.
    model = LGBMClassifier(
        max_depth=max_depth,
        num_leaves=num_leaves,
        min_child_samples=min_child_samples,
        colsample_bytree=colsample_bytree,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        random_state=61,
    )

    # K-Fold Cross validation을 구현합니다.
    folds = StratifiedKFold(n_splits=K, random_state=61, shuffle=True)
    losses = []

    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]

        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        loss = evaluation_metric(y_val, preds)
        losses.append(loss)


    # K-Fold의 평균 loss값을 돌려줍니다.
    return np.mean(losses)

In [35]:
K = 5   # Kfold 수
opt_func = partial(optimizer, X=X, y=y, K=K)

study = optuna.create_study(direction="maximize") # 최소/최대 어느 방향의 최적값을 구할 건지.
study.optimize(opt_func, n_trials=50)

[I 2023-10-15 01:24:34,215] A new study created in memory with name: no-name-032db553-1e11-401d-a0d6-5384561b33fb
[I 2023-10-15 01:27:50,984] Trial 0 finished with value: 0.6577500206037464 and parameters: {'max_depth': 19, 'num_leaves': 512, 'min_child_samples': 64, 'colsample_bytree': 0.6502067446158839, 'n_estimators': 1215, 'learning_rate': 0.017442666412207996}. Best is trial 0 with value: 0.6577500206037464.
[I 2023-10-15 01:27:57,779] Trial 1 finished with value: 0.663642404536692 and parameters: {'max_depth': 21, 'num_leaves': 32, 'min_child_samples': 68, 'colsample_bytree': 0.7272271257119367, 'n_estimators': 234, 'learning_rate': 0.06706234390276307}. Best is trial 1 with value: 0.663642404536692.
[I 2023-10-15 01:30:02,607] Trial 2 finished with value: 0.6509488588781851 and parameters: {'max_depth': 17, 'num_leaves': 512, 'min_child_samples': 77, 'colsample_bytree': 0.6598573369208828, 'n_estimators': 1613, 'learning_rate': 0.07823595927762073}. Best is trial 1 with value: 

In [36]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_colsample_bytree,params_learning_rate,params_max_depth,params_min_child_samples,params_n_estimators,params_num_leaves,state
0,0,0.657750,2023-10-15 01:24:34.218367,2023-10-15 01:27:50.984095,0 days 00:03:16.765728,0.650207,0.017443,19,64,1215,512,COMPLETE
1,1,0.663642,2023-10-15 01:27:50.985628,2023-10-15 01:27:57.778574,0 days 00:00:06.792946,0.727227,0.067062,21,68,234,32,COMPLETE
2,2,0.650949,2023-10-15 01:27:57.780574,2023-10-15 01:30:02.606383,0 days 00:02:04.825809,0.659857,0.078236,17,77,1613,512,COMPLETE
3,3,0.656836,2023-10-15 01:30:02.608656,2023-10-15 01:32:06.592852,0 days 00:02:03.984196,0.739758,0.018450,19,78,1397,512,COMPLETE
4,4,0.655885,2023-10-15 01:32:06.594879,2023-10-15 01:33:55.904361,0 days 00:01:49.309482,0.637825,0.027265,22,76,1337,2048,COMPLETE
5,5,0.654131,2023-10-15 01:33:55.905362,2023-10-15 01:34:42.350474,0 days 00:00:46.445112,0.763012,0.064846,18,79,851,1024,COMPLETE
6,6,0.653173,2023-10-15 01:34:42.352475,2023-10-15 01:35:29.342105,0 days 00:00:46.989630,0.614330,0.086541,18,65,734,1024,COMPLETE
7,7,0.657078,2023-10-15 01:35:29.344105,2023-10-15 01:36:47.156751,0 days 00:01:17.812646,0.697978,0.022347,15,76,1855,128,COMPLETE
8,8,0.654337,2023-10-15 01:36:47.158585,2023-10-15 01:37:39.198144,0 days 00:00:52.039559,0.788042,0.059847,16,61,1501,64,COMPLETE
9,9,0.606952,2023-10-15 01:37:39.200143,2023-10-15 01:37:44.874245,0 days 00:00:05.674102,0.653504,0.007628,23,63,159,32,COMPLETE


In [37]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

Best Score: 0.6656
Best params:  {'max_depth': 24, 'num_leaves': 32, 'min_child_samples': 74, 'colsample_bytree': 0.7080352104708246, 'n_estimators': 63, 'learning_rate': 0.09089244698723227}


In [38]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(study)

In [39]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

### 7. 테스트 및 제출 파일 생성

In [40]:
# Make KFold OOF prediction
def oof_preds(best_model):

    # make KFold
    folds = StratifiedKFold(n_splits=K, random_state=61, shuffle=True)
    final_preds = []
    losses = []
    oof = np.full(len(X), np.nan)
    # fitting with best_model
    for i, (train_idx, val_idx) in enumerate(folds.split(X, y)):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]

        print(f"========== Fold {i+1} ==========")
        best_model.fit(X_train, y_train)
        preds = best_model.predict_proba(X_val)[:, 1]
        oof[val_idx] = preds
        test_preds = best_model.predict_proba(test)[:, 1]
        final_preds.append(test_preds)
        loss = evaluation_metric(y_val, preds)

        losses.append(loss)

    avg_loss = np.mean(losses)
    print(f"Loss : {avg_loss:.4f}")
    return final_preds, oof

In [41]:
test.info() # 결측치 없음.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67842 entries, 0 to 67841
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 67842 non-null  int64  
 1   loc                67842 non-null  float64
 2   v(g)               67842 non-null  float64
 3   ev(g)              67842 non-null  float64
 4   iv(g)              67842 non-null  float64
 5   n                  67842 non-null  float64
 6   v                  67842 non-null  float64
 7   l                  67842 non-null  float64
 8   d                  67842 non-null  float64
 9   i                  67842 non-null  float64
 10  e                  67842 non-null  float64
 11  b                  67842 non-null  float64
 12  t                  67842 non-null  float64
 13  lOCode             67842 non-null  int64  
 14  lOComment          67842 non-null  int64  
 15  lOBlank            67842 non-null  int64  
 16  locCodeAndComment  678

In [42]:
## X_test 만들기 : 앞서했던 전처리를 동일하게 적용해주면 됨.


In [43]:
best_params = study.best_trial.params

# define best model
best_model = LGBMClassifier(**best_params, random_state=61)

# model finalization : 가장 일반적으로 좋은 예측 성능을 냈던 모델로, 전체 데이터 트레이닝.

preds, oof = oof_preds(best_model)
preds = np.mean(preds, axis=0)
preds

========== Fold 1 ==========
========== Fold 2 ==========
========== Fold 3 ==========
========== Fold 4 ==========
========== Fold 5 ==========
Loss : 0.7918


array([0.24119651, 0.18711343, 0.66155625, ..., 0.17619331, 0.10272604,
       0.80090445])

In [44]:
submission['defects'] = preds

In [45]:
submission.to_csv(base_path+'preds/lightgbm.csv', index=False)

In [46]:
oof_df = pd.DataFrame({
    'lightgbm_oof': oof
})
oof_df.head()

,lightgbm_oof
0,0.074083
1,0.055853
2,0.049316
3,0.080914
4,0.104143


In [47]:
oof_df.to_csv(base_path+'oof/lightgbm.csv', index=False)